In [3]:
import sqlite3

The data from the probe were written into a SQLite databse. We will read those back into the workspace as appropriate using SQLite syntax. In prep for that, we set up a connection to the database and a cursor for executing commands.

In [9]:
conn = sqlite3.connect('henry_constants.db')
c = conn.cursor()

In [12]:
c.execute("""SELECT AVERAGE(dissolved_oxygen), AVERAGE(temperature) FROM water_rt""")
water_rt = c.fetchall()
do_water, t_water = water_rt

In [20]:
c.execute("""SELECT name FROM sqlite_master WHERE type='table' AND name!='water_rt'""")
phantoms = c.fetchall()

9.186253101736973


In [ ]:
Hcc = 3.2e-2
R = 8.314  # JK-1mol-1
c.execute("""CREATE TABLE IF NOT EXISTS henry_constants_calculated (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        phantom STRING NOT NULL,
        average_dot FLOAT NOT NULL,
        hcxx FLOAT NOT NULL,
        khx FLOAT NOT NULL)
        """)
for phantom in phantoms:
    c.execute(f"""SELECT AVERAGE(dissolved_oxygen) FROM {phantom}""")
    do = c.fetchall()
    kHx = (Hcc / R * t_water) * (do / do_water)
    pO2 = do_water / kH

    c.execute(f"""INSERT INTO henry_constants_calculated (phantom, average_do, hccx, khx) VALUES (?, ?, ?, ?, ?)}""",
              (phantom, do, Hccx, kHx))
c.commit()